In [ ]:
import flwr as fl
import tensorflow as tf
import numpy as np
import os

# Load preprocessed data
X_train = np.load("processed_dataset/X_train.npy")
y_train = np.load("processed_dataset/y_train.npy")
X_test = np.load("processed_dataset/X_test.npy")
y_test = np.load("processed_dataset/y_test.npy")

# Build a simple CNN model
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(set(y_train)), activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Federated Learning client
class FLClient(fl.client.NumPyClient):
    def __init__(self):
        self.model = create_model()
    
    def get_parameters(self):
        return self.model.get_weights()
    
    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(X_train, y_train, epochs=1, batch_size=32, verbose=0)
        return self.model.get_weights(), len(X_train), {}
    
    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(X_test, y_test, verbose=0)
        return loss, len(X_test), {"accuracy": accuracy}

# Start FL client
if __name__ == "__main__":
    fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=FLClient())
